In [2]:
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt

In [3]:
# get all files from the data/ folder and concatenate them into a single dataframe
# we dont know the file names, so we use the glob module to get all files
files = glob.glob('data/*.csv')
dfs = [pd.read_csv(f) for f in files]
df = pd.concat(dfs)


block_number = len(df['block_number'].unique())
block_gas_prices = df.groupby('block_number')['gas_price'].apply(list)
min_gas_prices = block_gas_prices.apply(min).sort_index()

# Cross Validation Partition

In [4]:
from sklearn.model_selection import TimeSeriesSplit
import numpy as np

# Assuming 'min_gas_prices' is the time series data (pandas Series)
# Reserve the last 20% of the data for testing
test_size = int(len(min_gas_prices) * 0.2)
train_size = len(min_gas_prices) - test_size

# Split the data
train_data = min_gas_prices.iloc[:train_size]
test_data = min_gas_prices.iloc[train_size:]

# Use TimeSeriesSplit for cross-validation on the training data
n_splits = 5  # Number of cross-validation splits
tscv = TimeSeriesSplit(n_splits=n_splits)

for fold, (train_idx, val_idx) in enumerate(tscv.split(train_data)):
    train_fold = train_data.iloc[train_idx]
    val_fold = train_data.iloc[val_idx]
    print(f"Fold {fold + 1}")
    print("Train indices:", train_idx)
    print("Validation indices:", val_idx)
    print("Train data:", train_fold)
    print("Validation data:", val_fold)
    print()


Fold 1
Train indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 

# LSTM